In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch as t
import torch.nn as nn
import torch.nn.functional as F
from beartype import beartype as typed
from beartype.door import die_if_unbearable as assert_type
from datasets import load_dataset
from jaxtyping import Float, Int, Bool
from typing import Callable
from torch import Tensor as TT
from transformers import AutoModelForCausalLM, AutoTokenizer
from einops import einops as ein

%load_ext autoreload
%autoreload 2

In [ ]:
model_name = "Mlxa/brackets-nested"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from transformers import GPTNeoForCausalLM

In [238]:
from utils import layer_importance_on_last_token

result = layer_importance_on_last_token(
    model,
    tokenizer,
    real_prompt="<10 <11 <12 <12 <12 12> 12> 12> 11>",
    corrupted_prompt="<10 <11 <13 <13 <13 13> 13> 13> 11>",
)
for name, value in result.items():
    if abs(value) > 0.05:
        print(f"{name[12:]}:    \t{value:.2f}")

wte:    	0.43
h.0.attn:    	0.44
h.1.attn:    	0.17
h.1.mlp:    	-0.08
h.2.mlp:    	0.08
h.3.mlp:    	-0.06
h.4.attn:    	0.14
h.5.attn:    	0.10
h.5.mlp:    	-0.06
h.6.attn:    	0.28
h.7.attn:    	1.28
h.7.mlp:    	0.46


In [ ]:
from utils import show_patterns, prompt_from_template, get_losses
from IPython.display import display

# prompt = prompt_from_template("(((((())((()))))))", random=True)
# prompt = "<1 <2 <3 3> 2> 1> <9 <11 <12 <10 <11 <12 12> 11> 10> 12> 11> 9>"
prompt = "<10 <11 11> 10> <9 <11 11> 9> <8 <11 11> 8>"
print(get_losses(model, tokenizer, prompt))
display(show_patterns(model, tokenizer, prompt, layer=1))